## fine tuninig

世の中にはたくさんの学習済みのモデルがあります。それらを使ってfine tuningを行います  
今回はtensorflow_hubにある学習済みモデルを使います。

### 学習済みモデルの再利用

まずはモデルを読み込み、使用してみます。

In [ ]:
!sudo pip install -q -U tf-hub-nightly

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4  
今回はこのモデルを使用します。mobile_netは軽量で高速なモデルで、精度もそれなりに高く、非常に使いやすいモデルです。

In [ ]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
!pip install pillow

In [ ]:
import numpy as np
import PIL.Image as Image

image_url = "https://dol.ismcdn.jp/mwimgs/7/1/670m/img_71c53c1d81500a1cf73a4f543e72413f27838.jpg" # 自分で指定

img = tf.keras.utils.get_file('inu.jpg', image_url)
img = Image.open(img).resize(IMAGE_SHAPE)
img


In [ ]:
img = np.array(img) / 255.0
print(img.shape)

batch_sizeの分だけ次元を増やしてあげてから、predictしてみます。

In [ ]:
## <todo> predictを行ってみましょう
img = ___ ## batch_size部分の次元を合わせます (1,  244, 244, 3)
result = ___ ## predictを行います
predicted_class = np.___(___, axis=-1) ## 一番確率の高い要素を出力します
predicted_class

予測結果が得られたので、このclass_idがなんに紐づいているのか定義から確認してみます。

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
predicted_class_name = imagenet_labels[predicted_class]
print(predicted_class_name)

きちんと学習済みで推論できていることが確認できます。 
他にも何枚か試してみてください。

ロードしたモデルについても確認しておきます。

In [ ]:
classifier.summary()

Trainable params: 0 からこのモデルが再トレーニングできないモデルなことがわかります。

## 最終層の再学習
    
tensorflow_hubには完全に使用可能なモデルから、最終層やいくつかの層をあえて取り外し、特徴ベクトルを取り出せるようなモデルもあります。  
今回は再学習が目的なので、特徴量ベクトルだけ取り出し、最終層を再学習させます。

そのため、再度、特徴量ベクトルが取り出すことができるモデルを読み込みます。  
https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4  
ここからurlをコピーして使います。

In [ ]:
feature_vector_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

IMAGE_SHAPE = (224, 224)
NUM_CLASSES=2

model = tf.keras.Sequential([
    hub.KerasLayer(feature_vector_url, input_shape=IMAGE_SHAPE+(3,)),
    ## <todo> 最終層にunit数がNUM_CLASSES, activationがsoftmaxのDenseを追加してみましょう。
])
model.summary()

最終層につけたDenseがTrainableで、特徴量ベクトルの層がUntrainableになっています。

モデルの準備はできたので、学習データセットを用意します。

https://www.tensorflow.org/datasets/catalog/overview  
上記のカタログの中から、  
https://www.tensorflow.org/datasets/catalog/horses_or_humans  
こちらのデータセットを今回は使います。

In [ ]:
%%bash
TRAIN_DATA_URL=https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
VAL_DATA_URL=https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
curl -o train.zip ${TRAIN_DATA_URL}
curl -o val.zip ${VAL_DATA_URL}
rm -rf train
rm -rf val
mkdir -p train
mkdir -p val
unzip -qq train.zip -d train
unzip -qq val.zip -d val

In [ ]:
import keras 

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_image_data = train_image_generator.flow_from_directory("./train", target_size=IMAGE_SHAPE)

eval_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
eval_image_data = eval_image_generator.flow_from_directory("./val", target_size=IMAGE_SHAPE)

試しにclassifierで推論してみると、

In [ ]:
for image_batch, label_batch in train_image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  result = classifier.predict(image_batch)
  predicted_class = np.argmax(result[0], axis=-1)
  print(imagenet_labels[predicted_class])
  break

## Training

さてデータの準備ができたので、訓練を開始します。

In [ ]:
## <todo> optimizerがadam, lossがcategorical_crossentropyのモデルをコンパイルしてみましょう。
model.___(
  ___,
  ___,
  metrics=['categorical_accuracy'])

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['categorical_accuracy'])
    self.model.reset_metrics()
    
steps_per_epoch = np.ceil(train_image_data.samples/train_image_data.batch_size)

batch_stats_callback = CollectBatchStats()

# <todo> 適切な引数を入れてmodelのtrainingをしましょう。
history = model.fit_generator(generator=___, epochs=1,
                              steps_per_epoch=___,
                              validation_data=___,
                              callbacks = [batch_stats_callback])

### 可視化
訓練が終わったら、lossとaccuracyの推移を確認してみます。

In [ ]:
import matplotlib.pylab as plt

plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [ ]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)